In [ ]:

!pip install tensorflow transformers
import tensorflow as tf
from transformers import MobileBertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import numpy as np


In [ ]:
import random
from sklearn.model_selection import train_test_split
from transformers import MobileBertTokenizer
import tensorflow as tf

# Define the temperature ranges and their corresponding recommendations
temperature_ranges = [
    (0, 15, "Heating recommended"),
    (16, 25, "Temperature is acceptable"),
    (26, 50, "Cooling recommended")
]

# Function to generate a sentence based on temperature
def generate_sentence(temp):
    return f"The temperature is {temp}°C"

# Function to generate dataset
def generate_dataset(num_samples=1000):
    texts = []
    labels = []

    for _ in range(num_samples):
        # Randomly choose a temperature range
        temp_range = random.choice(temperature_ranges)

        # Randomly select a temperature within the chosen range
        temperature = random.randint(temp_range[0], temp_range[1])

        # Generate the sentence
        sentence = generate_sentence(temperature)
        label = temp_range[2]

        texts.append(sentence)
        labels.append(label)

    return texts, labels

# Generate the dataset
train_texts, train_labels = generate_dataset(num_samples=1000)  # You can increase this number
val_texts, val_labels = generate_dataset(num_samples=200)  # Validation set

# Load MobileBERT tokenizer
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

# Tokenize and encode the train and validation texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=32)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=32)

# Create a mapping from labels to numerical IDs
label2id = {label: i for i, label in enumerate(set(train_labels + val_labels))}
id2label = {i: label for label, i in label2id.items()}

# Convert text labels to numerical IDs
train_labels = [label2id[label] for label in train_labels]
val_labels = [label2id[label] for label in val_labels]

# Split the data into training and validation sets (already done during dataset creation)
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(16)

# Display a sample of the dataset
for i in range(5):
    print(f"Text: {train_texts[i]} -> Label: {train_labels[i]}")


Text: The temperature is 8°C -> Label: 1
Text: The temperature is 7°C -> Label: 1
Text: The temperature is 43°C -> Label: 2
Text: The temperature is 34°C -> Label: 2
Text: The temperature is 44°C -> Label: 2


In [ ]:
# Load the model
model = TFBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=len(label2id))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

You are using a model of type mobilebert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at google/mobilebert-uncased were not used when initializing TFBertForSequenceClassification: ['seq_relationship___cls', 'mobilebert', 'predictions___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['cl

In [ ]:
# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)

# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

Epoch 1/3
63/63 [==============================] - 171s 2s/step - loss: 0.9275 - accuracy: 0.5130 - val_loss: 0.1495 - val_accuracy: 1.0000
Epoch 2/3
63/63 [==============================] - 90s 1s/step - loss: 0.0286 - accuracy: 1.0000 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 3/3
13/13 [==============================] - 3s 211ms/step - loss: 0.0056 - accuracy: 1.0000
Validation Loss: 0.005589762702584267
Validation Accuracy: 1.0


In [ ]:
model.save("/content/saved_model")  # or use any other directory path you prefer


In [ ]:
import tensorflow as tf

# Path to the saved model (update this path to where your model is saved)
saved_model_dir = "/content/saved_model"

# Convert the saved model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Apply integer quantization for both weights and activations
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optimize the model size and latency
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # Use integer operations

# Provide representative dataset function for quantization
def representative_data_gen():
    # Provide a small sample dataset for quantization
    for data in val_dataset.take(100): # Take 100 batches from the validation dataset
        yield [data[0]['input_ids'], data[0]['attention_mask'], data[0]['token_type_ids']]
        # Yield the actual input tensors expected by MobileBERT: input_ids, attention_mask, token_type_ids

converter.representative_dataset = representative_data_gen

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = "/content/model_quantized.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Quantized TensorFlow Lite model is saved at {tflite_model_path}")


Quantized TensorFlow Lite model is saved at /content/model_quantized.tflite


In [ ]:
from google.colab import files
files.download("/content/model_quantized.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Path to the saved model (update this path to where your model is saved)
saved_model_dir = "/content/saved_model"

# Convert the saved model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# Convert the model
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = "/content/model.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"TensorFlow Lite model is saved at {tflite_model_path}")

TensorFlow Lite model is saved at /content/model.tflite


In [ ]:


from google.colab import files
files.download("/content/model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>